<a href="https://colab.research.google.com/github/goerlitz/nlp-classification/blob/main/notebooks/10kGNAD/colab/22_10kGNAD_simpletransformers_hyperparam_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Optimization with Weights & Biases Sweeps

https://simpletransformers.ai/docs/tips-and-tricks/#hyperparameter-optimization

## Prerequisites

### Need GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Jun 13 11:27:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Install Libraries

In [ ]:
# install transformers
!pip install -q --upgrade tqdm==4.47.0 >/dev/null
!pip install -q --upgrade transformers simpletransformers >/dev/null

# check installed version
!pip freeze | grep transformers
# simpletransformers==0.61.6
# transformers==4.6.1

ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.
simpletransformers==0.61.6
transformers==4.6.1


In [ ]:
import pandas as pd
from pathlib import Path
import os

from simpletransformers.classification import ClassificationModel
from transformers import AutoTokenizer
from transformers import logging

logging.set_verbosity_error()

# os.environ["WANDB_SILENT"] = "true"

### Download Data

Get the 10k German News Articles Dataset

In [ ]:
%env DIR=data

!mkdir -p $DIR
!wget -nc https://github.com/tblock/10kGNAD/blob/master/train.csv?raw=true -nv -O $DIR/train.csv
!wget -nc https://github.com/tblock/10kGNAD/blob/master/test.csv?raw=true -nv -O $DIR/test.csv
!ls -lAh $DIR | cut -d " " -f 5-

env: DIR=data
2021-06-13 11:27:41 URL:https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv [24405789/24405789] -> "data/train.csv" [1]
2021-06-13 11:27:43 URL:https://raw.githubusercontent.com/tblock/10kGNAD/master/test.csv [2755020/2755020] -> "data/test.csv" [1]

2.7M Jun 13 11:27 test.csv
 24M Jun 13 11:27 train.csv


## Import Data

Load training and test dataset

In [ ]:
data_dir = Path(os.getenv("DIR"))

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'

def read_csv_10kGNAD(filepath: Path, columns=["labels", "text"]) -> pd.DataFrame:
    """Load 10kGNAD csv file, handling its specific file format."""
    f = pd.read_csv(filepath, sep=";", quotechar="'", names=columns)
    return f

In [ ]:
train_df = read_csv_10kGNAD(data_dir / 'train.csv')
print(train_df.shape[0], 'articles')
display(train_df.head())

9245 articles


,labels,text
0,Sport,21-Jähriger fällt wohl bis Saisonende aus. Wie...
1,Kultur,"Erfundene Bilder zu Filmen, die als verloren g..."
2,Web,Der frischgekürte CEO Sundar Pichai setzt auf ...
3,Wirtschaft,"Putin: ""Einigung, dass wir Menge auf Niveau vo..."
4,Inland,Estland sieht den künftigen österreichischen P...


In [ ]:
test_df = read_csv_10kGNAD(data_dir / 'test.csv')
print(test_df.shape[0], 'articles')
display(test_df.head())

1028 articles


,labels,text
0,Wirtschaft,"Die Gewerkschaft GPA-djp lanciert den ""All-in-..."
1,Sport,Franzosen verteidigen 2:1-Führung – Kritische ...
2,Web,Neues Video von Designern macht im Netz die Ru...
3,Sport,23-jähriger Brasilianer muss vier Spiele pausi...
4,International,Aufständische verwendeten Chemikalie bei Gefec...


## Prepare for Model Training

Model Input Requirements:

* columns should be labeled `labels` and `text` (already done during import)
* labels must be int values starting at `0`

### Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_labels(train: pd.DataFrame, test: pd.DataFrame):
    le = LabelEncoder()

    train_labels = le.fit_transform(train.labels)
    test_labels = le.transform(test.labels)

    return train.assign(labels=train_labels), test.assign(labels=test_labels)

train_df, test_df = encode_labels(train_df, test_df)
display(train_df.head())

,labels,text
0,5,21-Jähriger fällt wohl bis Saisonende aus. Wie...
1,3,"Erfundene Bilder zu Filmen, die als verloren g..."
2,6,Der frischgekürte CEO Sundar Pichai setzt auf ...
3,7,"Putin: ""Einigung, dass wir Menge auf Niveau vo..."
4,1,Estland sieht den künftigen österreichischen P...


In [ ]:
# # map label to integers
# mapping_s = pd.Series(train_df.labels.value_counts().index)
# mapping_s

In [ ]:
# # replace labels with integers starting at 0
# train_df.labels.replace(mapping_s.values, mapping_s.index, inplace=True)
# test_df.labels.replace(mapping_s.values, mapping_s.index, inplace=True)
# display(train_df.head())
# display(test_df.head())

### Compute Class Weights

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

def class_weights(labels: pd.Series):
    uniq_labels = labels.unique()
    weights = compute_class_weight("balanced", uniq_labels, labels)
    return pd.Series(weights, index=uniq_labels).sort_index()

weights_s = class_weights(train_df.labels)
list(weights_s.values)

[1.7091883897208358,
 1.1251064865522697,
 0.7553104575163399,
 2.117983963344788,
 0.6802796173657101,
 0.9502518244423888,
 0.6807304322214859,
 0.8088363954505686,
 1.9907407407407407]

In [ ]:
import numpy as np

list(np.ones(train_df.labels.nunique()))

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

## Evaluation Setup

In [ ]:
import wandb

In [ ]:
sweep_config = {
    "method": "bayes",  # grid, random
    # "metric": {"name": "eval_loss", "goal": "minimize"},
    "metric": {"name": "f1", "goal": "maximize"},
    "parameters": {
        "num_train_epochs": {"values": [1, 2, 3, 4, 5]},
        "learning_rate": {"min": 1e-5, "max": 1e-4},
        "class_weights": {"values": [0, 1]},
        "train_batch_size": {"values": [8, 16, 24, 32]},
    },
}

project_name = "10kgnad_simple_sweep"

sweep_id = wandb.sweep(sweep_config, project=project_name)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: fub6v242
Sweep URL: https://wandb.ai/goerlitz/10kgnad_simple_sweep/sweeps/fub6v242


In [ ]:
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "evaluate_during_training": True,
             "fp16": False,
            #  "evaluate_during_training_verbose": False,
            #  "evaluate_during_training_silent": True,
             "wandb_project": project_name,
            #  "silent": True,
             }

In [ ]:
# model_args = ClassificationArgs()
# model_args.reprocess_input_data = True
# model_args.overwrite_output_dir = True
# model_args.evaluate_during_training = True
# model_args.manual_seed = 4
# model_args.use_multiprocessing = True
# model_args.train_batch_size = 16
# model_args.eval_batch_size = 8
# model_args.labels_list = ["true", "false"]
# model_args.wandb_project = "Simple Sweep"

In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')

def precision_multiclass(labels, preds):
    return precision_score(labels, preds, average='macro')

def recall_multiclass(labels, preds):
    return recall_score(labels, preds, average='macro')

In [ ]:
model_type = "distilbert"
model_name = "distilbert-base-german-cased"

def train():
    # Initialize a new wandb run
    wandb.init()

    # need to create a tokenizer first and adjust train args with lower case setting
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model_args = {**train_args, **{ "do_lower_case": tokenizer.do_lower_case }}

    # print(wandb.config["class_weights"])
    weight = None if wandb.config["class_weights"] == 0 else list(weights_s.values)

    # Create a ClassificationModel
    model = ClassificationModel(
        model_type,
        model_name,
        num_labels=train_df.labels.nunique(),
        weight=weight,
        args=model_args,
        sweep_config=wandb.config
    )

    # Train the model
    model.train_model(
        train_df,
        eval_df=test_df,
        verbose=False,
        show_running_loss=False,
        f1=f1_multiclass,
        acc=accuracy_score,
        precision=precision_multiclass,
        recall=recall_multiclass,
    )

    # Sync wandb
    wandb.join()

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: r4q40dtk with config:
wandb: 	class_weights: 0
wandb: 	learning_rate: 2.8040746524490095e-05
wandb: 	num_train_epochs: 4
wandb: 	train_batch_size: 8
wandb: Currently logged in as: goerlitz (use `wandb login --relogin` to force relogin)


Training loss,0.00643
lr,0.0
global_step,4624
_runtime,395
_timestamp,1623584064
_step,97
mcc,0.87294
train_loss,0.00079
eval_loss,0.58165
f1,0.88449
acc,0.88911


Training loss,█▇▆▁▁▄▅▃▂▅▁▂▁▅▂▂▁▂▄▂▁▁▂▁▁▁▂▄▃▁▁▁▁▁▁▁▁▁▁▁
lr,▂▅▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
mcc,▁▄▅█▅▇
train_loss,▁▃▆█▁▁
eval_loss,▁▁▄▅██
f1,▁▆▆█▆▇
acc,▁▅▅█▆▇


wandb: Agent Starting Run: lpcup7ew with config:
wandb: 	class_weights: 0
wandb: 	learning_rate: 4.803912806124468e-05
wandb: 	num_train_epochs: 1
wandb: 	train_batch_size: 32


Training loss,0.14985
lr,1e-05
global_step,289
_runtime,86
_timestamp,1623584155
_step,5
mcc,0.84502
train_loss,0.4339
eval_loss,0.40834
f1,0.86066
acc,0.86479


...

Training loss,█▅▄▃▃▂▄▂▁▄▃▂▂▄▂▂▁▁▁▂▁▁▁
lr,▆██▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▁▁
global_step,▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇███
_runtime,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇██
_timestamp,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇██
_step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
mcc,▁▃█
train_loss,▁█▁
eval_loss,▁▅█
f1,▁▅█
acc,▁▃█


wandb: Agent Starting Run: hrwf1wpj with config:
wandb: 	class_weights: 1
wandb: 	learning_rate: 4.074322869540972e-05
wandb: 	num_train_epochs: 3
wandb: 	train_batch_size: 16


Training loss,0.2072
lr,0.0
global_step,1734
_runtime,242
_timestamp,1623619538
_step,36
mcc,0.87973
train_loss,0.00768
eval_loss,0.41149
f1,0.89198
acc,0.89494


Training loss,█▄▃▄▃▃▃▂▃▂▃▂▁▂▃▂▄▂▂▂▂▃▁▂▁▁▂▁▁▁▁▂▁▂
lr,▄████▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
global_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
_step,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
mcc,▁▆█
train_loss,█▇▁
eval_loss,▁▃█
f1,▁▆█
acc,▁▆█


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/internal/internal_api.py", line 1233, in agent_heartbeat
    timeout=60,
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/lib/retry.py", line 102, in __call__
    result = self._call_fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/sdk/internal/internal_api.py", line 127, in execute
    return self.client.execute(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/vendor/gql-0.2.0/gql/client.py", line 52, in execute
    result = self._get_result(document, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/vendor/gql-0.2.0/gql/client.py", line 60, in _get_result
    return self.transport.execute(document, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/wandb/vendor/gql-0.2.0/gql/transport/requests.py", line 38, in execute
    request = requests.post(self.url, **post_args)
  File "/u